In [2]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
import regex as re

from sklearn.model_selection import train_test_split
from tqdm import tqdm

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from IPython.display import clear_output

## Train

In [11]:
df = pd.read_csv('../../data/for_linear_models/train_feature.csv' , header=0)#,  on_bad_lines='skip')
df

,compiller,version,compiler_option,name_hash,instructions_count,func_body,operators,operands,func_class
0,gcc,10.2.1,Og,9c9ff5709191ddfb3571924ab4813c61,12,"['mov rdi, rbx', 'call pthreadFCN', 'cmp eax, ...",mov call cmp je,rdi rbx pthreadFCN eax 4 methodMETHOD,System
1,gcc,9.3.0,Ofast,17409f6679b9f51ee8aa08433f474ba0,6,"['test rdi, rdi', 'je ADDR', 'mov esi, ADDR', ...",test je mov jmp nop ret,rdi rdi ADDR esi ADDR FCN,System
2,gcc,9.3.0,Ofast,154dd4eaaebee58e4dba5db9d173cbb4,1,['ret'],ret,NaN,Another
3,gcc,9.3.0,Ofast,1f3246ca1773bad276222413af9bb16a,602,"['push r15', 'lea r15, [ADDR]', 'push r14', 'm...",push lea push mov push push push push sub mov ...,r15 r15 [ADDR] r14 r14 rdi r13 r12 rbp rbx rsp...,Another
4,gcc,9.3.0,Ofast,f96255f164e19a3b0ca3891d7319f305,406,"['push r15', 'push r14', 'push r13', 'mov r13,...",push push push mov push push mov push sub mov ...,r15 r14 r13 r13 rdi r12 rbp rbp rsi rbx rsp AD...,Another
...,...,...,...,...,...,...,...,...,...
14432,gcc,9.3.0,Ofast,9b23f4fed5f5903a26f38682bab610b7,176,"['push r15', 'mov ecx, ADDR', 'push r14', 'pus...",push mov push push mov push push mov push mov ...,r15 ecx ADDR r14 r13 r13 rdi r12 rbp rbp rsi r...,System
14433,gcc,10.2.1,Og,e62bd824cf79f10b7d42a975bb4a6e5a,6,"['mov rdx, qword [rdi + ADDR]', 'mov r8d, 6', ...",mov mov add xor lea jmp,rdx qword [rdi + ADDR] r8d 6 rdi ADDR esi esi ...,System
14434,gcc,10.2.1,Og,904fc189b64f63ffe86fb5d7eec010b3,26,"['mov edi, ADDR', 'call FCN', 'mov r12, rax', ...",mov call mov call lea mov mov mov call lea lea...,edi ADDR FCN r12 rax FCN rcx [ADDR] esi 1 rdi ...,Another
14435,gcc,9.3.0,Ofast,6d69aeed11dbabd26fa3dd7d509db815,195,"['push r15', 'lea rax, [ADDR]', 'push r14', 'l...",push lea push lea push push push mov add push ...,r15 rax [ADDR] r14 r14 [rdi + 8] r13 r12 rbp r...,Another


In [12]:
y_train = df["func_class"]

In [ ]:
vec = CountVectorizer(analyzer= 'word', ngram_range = (1, 3))
bow = vec.fit_transform(df["func_body"])

In [ ]:
# + GRAPH

In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = int(0.75 * X.shape[0])
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]
X_train.shape, X_test.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0xE2E4)
X_train.shape, X_val.shape

In [ ]:
device = torch.device('cpu')
device

In [ ]:
def plot_losses(train_losses, train_metrics, val_losses, val_metrics):
    '''
    Plot losses and metrics while training
      - train_losses: sequence of train losses
      - train_metrics: sequence of train MSE values
      - val_losses: sequence of validation losses
      - val_metrics: sequence of validation MSE values
    '''
    clear_output()
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].plot(range(1, len(train_losses) + 1), train_losses, label='train')
    axs[0].plot(range(1, len(val_losses) + 1), val_losses, label='val')
    axs[1].plot(range(1, len(train_metrics) + 1), train_metrics, label='train')
    axs[1].plot(range(1, len(val_metrics) + 1), val_metrics, label='val')

    if max(train_losses) / min(train_losses) > 10:
        axs[0].set_yscale('log')

    if max(train_metrics) / min(train_metrics) > 10:
        axs[0].set_yscale('log')

    for ax in axs:
        ax.set_xlabel('epoch')
        ax.legend()

    axs[0].set_ylabel('loss')
    axs[1].set_ylabel('MSE')
    plt.show()


def train_and_validate(model, optimizer, criterion, metric, train_loader, val_loader,
                       num_epochs, verbose=True):
    '''
    Train and validate neural network
      - model: neural network to train
      - optimizer: optimizer chained to a model
      - criterion: loss function class
      - metric: function to measure MSE taking neural networks predictions
                 and ground truth labels
      - train_loader: DataLoader with train set
      - val_loader: DataLoader with validation set
      - num_epochs: number of epochs to train
      - verbose: whether to plot metrics during training
    Returns:
      - train_mse: training MSE over the last epoch
      - val_mse: validation MSE after the last epoch
    '''
    train_losses, val_losses = [], []
    train_metrics, val_metrics = [], []

    for epoch in range(1, num_epochs + 1):
        model.train()
        running_loss, running_metric = 0, 0
        pbar = tqdm(train_loader, desc=f'Training {epoch}/{num_epochs}') \
            if verbose else train_loader
 

        for i, (X_batch, y_batch) in enumerate(pbar, 1):
            '''
            YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
            Do forward and backward passes
            predictions = ...
            loss = ...
            '''
 
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
#             print("===X====")
#             print(X_batch)
#             print("====Y===")
#             print(y_batch)
#             print("====Pred===")

            optimizer.zero_grad()
            # images: batch_size x num_channels x height x width
            predictions = model(X_batch)
#             print(predictions)
#             print("=======")
            
            # logits: batch_size x num_classes
            loss = criterion(predictions, y_batch)
            loss.backward()
            optimizer.step()

            
            
           # print(predictions.shape)
           # print(y_batch.shape)
            
            
            

            with torch.no_grad():
                metric_value = metric(predictions, y_batch)
                if type(metric_value) == torch.Tensor:
                    metric_value = metric_value.item()
                running_loss += loss.item() * X_batch.shape[0]
                running_metric += metric_value * X_batch.shape[0]

            if verbose and i % 100 == 0:
                pbar.set_postfix({'loss': loss.item(), 'MSE': metric_value})

        train_losses += [running_loss / len(train_loader.dataset)]
        train_metrics += [running_metric / len(train_loader.dataset)]

        model.eval()
        running_loss, running_metric = 0, 0
        pbar = tqdm(val_loader, desc=f'Validating {epoch}/{num_epochs}') \
            if verbose else val_loader

        for i, (X_batch, y_batch) in enumerate(pbar, 1):
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)
            with torch.no_grad():
                '''
                YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
                Do evaluation
                predictions = ...
                loss = ...
                '''
                predictions = model(X_batch)
                #print(type(predictions))
                #exit(0)
                # logits: batch_size x num_classes
                loss = criterion(predictions, y_batch)
                
                #print(predictions.shape)
                #print(y_batch.shape)

                metric_value = metric(predictions, y_batch)
                if type(metric_value) == torch.Tensor:
                    metric_value = metric_value.item()
                running_loss += loss.item() * X_batch.shape[0]
                running_metric += metric_value * X_batch.shape[0]

            if verbose and i % 100 == 0:
                pbar.set_postfix({'loss': loss.item(), 'MSE': metric_value})

        val_losses += [running_loss / len(val_loader.dataset)]
        val_metrics += [running_metric / len(val_loader.dataset)]

        if verbose:
            plot_losses(train_losses, train_metrics, val_losses, val_metrics)
    
    if verbose:
        print(f'Validation MSE: {val_metrics[-1]:.3f}')

    return train_metrics[-1], val_metrics[-1]

In [ ]:
# from torch.utils.data import DataLoader, TensorDataset
# X_train = X_train.astype(np.float32)
# X_val = X_val.astype(np.float32)

# train_set = torch.utils.data.TensorDataset(torch.from_numpy(X_train ) , torch.from_numpy(le.transform(y_train)))
# train_loader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=2, pin_memory=False)
# val_set = torch.utils.data.TensorDataset(torch.from_numpy(X_val), torch.from_numpy(le.transform(y_val)))
# val_loader = DataLoader(val_set, batch_size=64, shuffle=False, num_workers=2, pin_memory=False)

In [ ]:
MIN = np.min(le.transform(y_train))
MAX = np.max(le.transform(y_train))
MIN_MAX = MAX - MIN

def normalize(sample):
    '''
    Min-max normalization to convert sample to [0, 1] range
    '''
    # YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
    return np.array(list(map( lambda x: (x - MIN) / MIN_MAX, sample  )))


def denormalize(sample):
    '''
    Denormalize sample from [0, 1] to initial range
    '''
    # YOUR CODE HERE (－.－)...zzzZZZzzzZZZ
    return np.array(list(map( lambda x: x* MIN_MAX + MIN , sample  )))  


In [ ]:
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
stdsc.fit(X_train)

X_train = stdsc.transform(X_train).astype(np.float32)
X_val = stdsc.transform(X_val).astype(np.float32)


y_train_tensor = torch.from_numpy(normalize(le.transform(y_train))).reshape(-1, 1)
y_train_tensor = y_train_tensor.float()
y_val_set_tensor =  torch.from_numpy(normalize(le.transform(y_val))).reshape(-1, 1)
y_val_set_tensor = y_val_set_tensor.float()

train_set = torch.utils.data.TensorDataset(torch.from_numpy(X_train ) , y_train_tensor )

train_loader = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=2, pin_memory=False)
val_set = torch.utils.data.TensorDataset(torch.from_numpy(X_val),y_val_set_tensor )
val_loader = DataLoader(val_set, batch_size=32, shuffle=False, num_workers=2, pin_memory=False)

In [ ]:
class HW_NET_LONG_DROP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(in_features= 90, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=128, out_features=128),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=128, out_features=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.model(x)

In [ ]:
num_epochs = 10
model_7 = HW_NET_LONG_DROP().to(device)
optimizer = torch.optim.Adam(model_7.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
#torch.optim.SGD(model_5.parameters(), lr=1e-5, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
criterion = nn.MSELoss().to(device)


train_mse, val_mse = train_and_validate(model=model_7, optimizer=optimizer, criterion=criterion,
                   metric = lambda y_pred, y: torch.mean((y - y_pred)**2) ,
                   train_loader=train_loader, val_loader = val_loader,
                       num_epochs=num_epochs, verbose=True)

## Test

In [104]:
df_test = pd.read_csv('../../data/for_linear_models/test.csv' , header=0)#,  on_bad_lines='skip')
df_test

,compiller,version,compiler_option,name_hash,instructions_count,func_body,func_class
0,gcc,9.3.0,Ofast,c0a1c3d0f5693db431cf0af91a6cf5c8,222,"push r15', 'push r14', 'push r13', 'push r12',...",'Data structures'
1,gcc,9.3.0,Ofast,f6107b511a9a5d14f546840ab4f977c1,235,"push r15', 'push r14', 'push r13', 'push r12',...",System
2,gcc,10.2.1,Og,078b20586a67d3bd8640d912116986af,141,"push r14', 'lea rax, [ADDR]', 'push r13', 'pus...",Another
3,gcc,10.2.1,Og,b7a12725e3bda88dbc8659d995d871b0,43,"push r12', 'push rbp', 'push rbx', 'mov rcx, q...",System
4,gcc,9.3.0,Ofast,c3adbe2561228077a80f22185bf2f665,3,"mov esi, ADDR', 'jmp FCN', 'nop word [rax + rax]",Programming
...,...,...,...,...,...,...,...
3072,gcc,9.3.0,Ofast,b1f95b629c3a388f71dd3a907d46f5d3,24,"test rsi, rsi', 'je ADDR', 'push r12', 'mov r1...",'Data structures'
3073,gcc,10.2.1,Og,1e75530aa7839e80d54dcf39df19f73b,217,"push r12', 'lea r12, [ADDR]', 'lea rdi, [ADDR]...",Another
3074,gcc,10.2.1,Og,d3acac61f322edec5d85849c5f4275dc,15,"push rbx', 'mov rbx, qword [rdi + ADDR]', 'mov...",Programming
3075,gcc,9.3.0,Ofast,6514518732c7b6e1a8347db83a9a98d4,8,"mov rdi, r13', 'call FCN', 'lea rax, [ADDR]', ...",System
